# General Overview - Data Wrangling

The dataset represents 10 years (1999-2008) of clinical care at 130 US hospitals and integrated delivery networks. It includes over 50 features representing patient and hospital outcomes. Information was extracted from the database for encounters that satisfied the following criteria.

- It is an inpatient encounter (a hospital admission).
- It is a diabetic encounter, that is, one during which any kind of diabetes was entered to the system as a diagnosis.
- The length of stay was at least 1 day and at most 14 days.
- Laboratory tests were performed during the encounter.
- Medications were administered during the encounter.

The data contains such attributes as patient number, race, gender, age, admission type, time in hospital, medical specialty of admitting physician, number of lab test performed, HbA1c test result, diagnosis, number of medication, diabetic medications, number of outpatient, inpatient, and emergency visits in the year before the hospitalization, etc.*

*Taken from [UC Irvine's Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Diabetes+130-US+hospitals+for+years+1999-2008)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = pd.read_csv('diabetic_data.csv', na_values=["?"], low_memory=False) # import data
# csv contains "?" for unknown values

diabetes = data.copy() # save a copy of data as diabetes

In [3]:
diabetes.head(10)

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),NaN,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),NaN,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),NaN,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),NaN,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),NaN,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
5,35754,82637451,Caucasian,Male,[50-60),NaN,2,1,2,3,...,No,Steady,No,No,No,No,No,No,Yes,>30
6,55842,84259809,Caucasian,Male,[60-70),NaN,3,1,2,4,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
7,63768,114882984,Caucasian,Male,[70-80),NaN,1,1,7,5,...,No,No,No,No,No,No,No,No,Yes,>30
8,12522,48330783,Caucasian,Female,[80-90),NaN,2,1,4,13,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
9,15738,63555939,Caucasian,Female,[90-100),NaN,3,3,4,12,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [4]:
diabetes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 50 columns):
encounter_id                101766 non-null int64
patient_nbr                 101766 non-null int64
race                        99493 non-null object
gender                      101766 non-null object
age                         101766 non-null object
weight                      3197 non-null object
admission_type_id           101766 non-null int64
discharge_disposition_id    101766 non-null int64
admission_source_id         101766 non-null int64
time_in_hospital            101766 non-null int64
payer_code                  61510 non-null object
medical_specialty           51817 non-null object
num_lab_procedures          101766 non-null int64
num_procedures              101766 non-null int64
num_medications             101766 non-null int64
number_outpatient           101766 non-null int64
number_emergency            101766 non-null int64
number_inpatient            101766 non

# Column Descriptions

Below is a list of all 50 columns, their data type, description, and possible values. The table is from the research article [Impact of HbA1c Measurement on Hospital Readmission Rates: Analysis of 70,000 Clinical Database Patient Records](https://www.hindawi.com/journals/bmri/2014/781670/tab1/), which used a larger dataset from which this one is taken from.

| Feature name | Type | Description | Values |
|-------------|------|------------------------|----------|
| Encounter ID | Numeric | Unique identifier of an encounter |
| Patient number | Numeric | Unique identifier of a patient |
| Race | Nominal | | Caucasian, Asian, African American, Hispanic, and other |
| Gender | Nominal | | male, female, and unknown/invalid |
| Age | Nominal | Grouped in 10-year intervals |
| Weight | Numeric | Weight in pounds |
| Admission type | Nominal | Integer identifier corresponding to 9 distinct values | For example: emergency, urgent, elective, newborn, and not available |
| Discharge disposition | Nominal | Integer identifier corresponding to 29 distinct values | For example: discharged to home, expired, and not available |
| Admission source | Nominal | Integer identifier corresponding to 21 distinct values | For example: physician referral, emergency room, and transfer from a hospital |
| Time in hospital | Numeric | Integer number of days between admission and discharge |
| Payer code | Nominal | Integer identifier corresponding to 23 distinct values | For example: Blue Cross/Blue Shield, Medicare, and self-pay |
| Medical specialty | Nominal | Integer identifier of a specialty of the admitting physician, corresponding to 84 distinct values | For example: cardiology, internal medicine, family/general practice, and surgeon |
| Number of lab procedures | Numeric | Number of lab tests performed during the encounter |
| Number of procedures | Numeric | Number of procedures (other than lab tests) performed during the encounter |
| Number of medications | Numeric | Number of distinct generic names administered during the encounter |
| Number of outpatient visits | Numeric | Number of outpatient visits of the patient in the year preceding the encounter |
| Number of emergency visits | Numeric | Number of emergency visits of the patient in the year preceding the encounter |
| Number of inpatient visits | Numeric | Number of inpatient visits of the patient in the year preceding the encounter |
| Diagnosis 1 | Nominal | The primary diagnosis (coded as first three digits of ICD9) | 848 distinct values |
| Diagnosis 2 | Nominal | Secondary diagnosis (coded as first three digits of ICD9) | 923 distinct values |
| Diagnosis 3 | Nominal | Additional secondary diagnosis (coded as first three digits of ICD9) | 954 distinct values |
| Number of diagnoses | Numeric | Number of diagnoses entered to the system |
| Glucose serum test result | Nominal | Indicates the range of the result or if the test was not taken | ">200," ">300," "normal," and "none" if not measured |
| A1c test result | Nominal | Indicates the range of the result or if the test was not taken | ">8" if the result was greater than 8%, ">7" if the result was greater than 7% but less than 8%, "normal" if the result was less than 7%, and "none" if not measured |
| Change of medications | Nominal | Indicates if there was a change in diabetic medications (either dosage or generic name) | "change" and "no change" |
| Diabetes medications | Nominal | Indicates if there was any diabetic medication prescribed | "yes" and "no" |
| 24 features for medications | Nominal | For the generic names: metformin, repaglinide, nateglinide, chlorpropamide, glimepiride, acetohexamide, glipizide, glyburide, tolbutamide, pioglitazone, rosiglitazone, acarbose, miglitol, troglitazone, tolazamide, examide, sitagliptin, insulin, glyburide-metformin, glipizide-metformin, glimepiride-pioglitazone, metformin-rosiglitazone, and metformin-pioglitazone, the feature indicates whether the drug was prescribed or there was a change in the dosage |  "up" if the dosage was increased during the encounter, "down" if the dosage was decreased, "steady" if the dosage did not change, and "no" if the drug was not prescribed |
| Readmitted | Nominal | Days to inpatient readmission |  "<30" if the patient was readmitted in less than 30 days, ">30" if the patient was readmitted in more than 30 days, and "No" for no record of readmission |

# Look For and Drop Duplicates

In [5]:
diabetes = diabetes.drop_duplicates()

diabetes.shape # no duplicates detected!

(101766, 50)

# Drop Irrelevant Columns

The majority of patients do not have a weight listed so this column can be dropped. Payer code and medical specialty are also missing for about half of all patients, but we do not need to know how the patients paid for their treatments or where they went for it. We don't need the encounter ID because there are no duplicates in the data, which suggests that each visit is unique.

In [7]:
# columns to drop
drop_cols = ['encounter_id', 'weight', 'payer_code', 'medical_specialty']

diabetes = diabetes.drop(columns=drop_cols)

In [8]:
diabetes.columns # confirm drop

Index(['patient_nbr', 'race', 'gender', 'age', 'admission_type_id',
       'discharge_disposition_id', 'admission_source_id', 'time_in_hospital',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted'],
      dtype='object')

# Addressing NaN Values + Removing Rows

The majority of columns are not missing any values. Only race, diagnosis 1, 2 and 3 contain missing values. Since there is no way to know the race of the patient using existing information, the best option is to remove the rows with a missing string for race.

In [9]:
diabetes.isnull().sum()

patient_nbr                    0
race                        2273
gender                         0
age                            0
admission_type_id              0
discharge_disposition_id       0
admission_source_id            0
time_in_hospital               0
num_lab_procedures             0
num_procedures                 0
num_medications                0
number_outpatient              0
number_emergency               0
number_inpatient               0
diag_1                        21
diag_2                       358
diag_3                      1423
number_diagnoses               0
max_glu_serum                  0
A1Cresult                      0
metformin                      0
repaglinide                    0
nateglinide                    0
chlorpropamide                 0
glimepiride                    0
acetohexamide                  0
glipizide                      0
glyburide                      0
tolbutamide                    0
pioglitazone                   0
rosiglitaz

In [25]:
# remove rows where race is null
diabetes = diabetes.dropna(axis=0, subset=['race'])

diabetes.shape

(99493, 46)

In [26]:
diabetes.isnull().sum()

patient_nbr                    0
race                           0
gender                         0
age                            0
admission_type_id              0
discharge_disposition_id       0
admission_source_id            0
time_in_hospital               0
num_lab_procedures             0
num_procedures                 0
num_medications                0
number_outpatient              0
number_emergency               0
number_inpatient               0
diag_1                        19
diag_2                       336
diag_3                      1349
number_diagnoses               0
max_glu_serum                  0
A1Cresult                      0
metformin                      0
repaglinide                    0
nateglinide                    0
chlorpropamide                 0
glimepiride                    0
acetohexamide                  0
glipizide                      0
glyburide                      0
tolbutamide                    0
pioglitazone                   0
rosiglitaz

After removing the rows without information in the race column, we are down to three columns with missing information: diagnosis 1, 2, and 3. Diagnosis 1 is described as the primary diagnosis made during the patient's visit while diagnosis 2 is the second and 3 is an any additional diagnoses made after that. Looking at the patients' rows that are missing a primary diagnosis, most of them have a second diagnosis or even a third. Since it doesn't make sense to have a second (or third) but not a primary diagnosis, we will remove these columns from the dataset.

The number of diagnoses column shows the total number of conditions a patient is diagnosed with. Only the first three are recorded, so those that are missing the first diagnosis but still a second or third are in error.

In [27]:
diabetes[['diag_1', 'diag_2', 'diag_3','number_diagnoses']][diabetes.diag_1.isnull()]

,diag_1,diag_2,diag_3,number_diagnoses
518,NaN,780,997,4
1267,NaN,250.82,401,5
1488,NaN,276,594,8
3197,NaN,250.01,428,7
14503,NaN,276,250.01,6
19714,NaN,112,585,8
32514,NaN,998,427,9
37693,NaN,780,295,9
49516,NaN,707,427,8
56931,NaN,585,427,8


In [29]:
# remove rows where diagnosis 1 is missing

diabetes = diabetes.dropna(axis=0, subset=['diag_1'])

diabetes.shape

(99474, 46)

There are two remaining diagnosis columns with missing values. Each number correlates to a specific condition so if there is a missing value, then it is likely that the patient only has one diagnosed condition. The number of diagnoses column lists the total number of diagnosed conditions. When looking at all three diagnosis columns, if the number is one, then diagnosis 2 and 3 can be filled in with a 0 to show that there is no additional diagnosis. If diagnosis 2 or 3 is missing a value and the number of diagnoses is greater than one, then some diagnoses were not recorded and the rows should be removed.

In [30]:
diabetes[['diag_1', 'diag_2', 'diag_3']].isnull().sum()

diag_1       0
diag_2     335
diag_3    1348
dtype: int64

In [46]:
diabetes[['diag_1','diag_2', 'diag_3','number_diagnoses']][diabetes.diag_2.isnull() & (diabetes.number_diagnoses > 1)].sort_values(by=['number_diagnoses'])

/Users/annatang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,diag_1,diag_2,diag_3,number_diagnoses
26220,250.81,NaN,NaN,2
86140,491,NaN,NaN,3
31671,250.82,NaN,NaN,3
76321,414,NaN,250,3
35105,996,NaN,250,3
...,...,...,...,...
68488,780,NaN,153,8
49892,276,NaN,250.6,8
20289,402,NaN,425,9
91199,440,NaN,274,9


In [48]:
# remove rows where diagnosis 2 is missing and number of diagnoses is greater than 1
diag_2 = diabetes[(diabetes.diag_2.isnull()) & (diabetes.number_diagnoses > 1)].index
 
diabetes.drop(diag_2, inplace=True)

In [51]:
# remaining rows with missing diagnosis 2 should all have one diagnosed condition
diabetes[['diag_1','diag_2', 'diag_3','number_diagnoses']][diabetes.diag_2.isnull()].sort_values(by=['number_diagnoses'])

,diag_1,diag_2,diag_3,number_diagnoses
0,250.83,NaN,NaN,1
25847,250.02,NaN,NaN,1
25945,250.02,NaN,NaN,1
28758,250.1,NaN,NaN,1
28932,250.01,NaN,NaN,1
...,...,...,...,...
12850,250.02,NaN,NaN,1
13022,250.13,NaN,NaN,1
13920,250.01,NaN,NaN,1
14215,250.03,NaN,NaN,1


In [53]:
diabetes[['diag_1','diag_2', 'diag_3']].isnull().sum()

diag_1       0
diag_2     214
diag_3    1300
dtype: int64

Diagnosis 3 is the last column left with unaccounted missing values. Since some patients have 1 or 2 diagnosed conditions, the diagnosis 3 column is left intentionally blank. The goal here is to remove the rows that have a diagnoses number greater than two.

In [58]:
# list of affected rows
diabetes[['diag_1','diag_2', 'diag_3', 'number_diagnoses']][diabetes.diag_3.isnull() & (diabetes.number_diagnoses > 2)].sort_values(by='number_diagnoses')

,diag_1,diag_2,diag_3,number_diagnoses
339,722,729,NaN,3
88159,820,250.02,NaN,3
76149,486,250.81,NaN,3
54990,496,250,NaN,3
26895,865,599,NaN,3
...,...,...,...,...
73165,414,413,NaN,8
68985,599,787,NaN,8
58558,491,427,NaN,8
101560,590,276,NaN,8


In [59]:
# remove rows with missing diagnosis 3 and number of diagnoses is greater than 2
diag_3 = diabetes[(diabetes.diag_3.isnull()) & (diabetes.number_diagnoses > 2)].index
 
diabetes.drop(diag_3, inplace=True)

In [60]:
# remaining rows with missing diagnosis 3
diabetes[['diag_1','diag_2', 'diag_3','number_diagnoses']][diabetes.diag_3.isnull()].sort_values(by=['number_diagnoses'])

,diag_1,diag_2,diag_3,number_diagnoses
0,250.83,NaN,NaN,1
17065,250.03,NaN,NaN,1
16851,250.03,NaN,NaN,1
16681,250.13,NaN,NaN,1
16641,250.03,NaN,NaN,1
...,...,...,...,...
19199,820,250,NaN,2
19243,250.01,305,NaN,2
19276,250.22,34,NaN,2
3072,722,250.01,NaN,2


In [70]:
diabetes.isnull().sum()

patient_nbr                    0
race                           0
gender                         0
age                            0
admission_type_id              0
discharge_disposition_id       0
admission_source_id            0
time_in_hospital               0
num_lab_procedures             0
num_procedures                 0
num_medications                0
number_outpatient              0
number_emergency               0
number_inpatient               0
diag_1                         0
diag_2                       214
diag_3                      1196
number_diagnoses               0
max_glu_serum                  0
A1Cresult                      0
metformin                      0
repaglinide                    0
nateglinide                    0
chlorpropamide                 0
glimepiride                    0
acetohexamide                  0
glipizide                      0
glyburide                      0
tolbutamide                    0
pioglitazone                   0
rosiglitaz

In [ ]:
# replace NaN with 0 in diagnosis 2 and 3 to show there is no additional diagnosis


# Unique Values per Column

Investigate the unique values of each column and look for error entries.

In [84]:
for x in diabetes.columns:
    print('Column Name: ' + x)
    print(diabetes[x].unique())

Column Name: patient_nbr
[  8222157  55629189  86047875 ... 140199494 120975314 175429310]
Column Name: race
['Caucasian' 'AfricanAmerican' 'Other' 'Asian' 'Hispanic']
Column Name: gender
['Female' 'Male' 'Unknown/Invalid']
Column Name: age
['[0-10)' '[10-20)' '[20-30)' '[30-40)' '[40-50)' '[50-60)' '[60-70)'
 '[70-80)' '[80-90)' '[90-100)']
Column Name: admission_type_id
[6 1 2 3 4 5 8 7]
Column Name: discharge_disposition_id
[25  1  3  6  2  5 11  7 10  4 14 18  8 13 12 16 17 22 23  9 20 15 24 28
 19 27]
Column Name: admission_source_id
[ 1  7  2  4  5  6 20  3 17  8  9 14 10 22 11 25 13]
Column Name: time_in_hospital
[ 1  3  2  4  5 13 12  9  7 10 11  6  8 14]
Column Name: num_lab_procedures
[ 41  59  11  44  51  31  70  73  68  33  47  62  60  55  49  75  45  29
  35  42  19  64  25  53  52  87  27  37  46  28  36  48  72  10   2  65
  67  40  58  57  32  83  34  39  69  38  56  22  96  78  61  88  66  43
  50   1  18  82  54   9  63  24  71  77  81  76  90  93   3 103  13  80
  85

In [87]:
# remove row where gender is Unknown/Invalid
gender = diabetes[diabetes.gender == 'Unknown/Invalid'].index

diabetes.drop(gender, inplace=True)

In [88]:
# confirm removal
diabetes.gender.unique()

array(['Female', 'Male'], dtype=object)

# Saving Cleaned Data

In [ ]:
# save cleaned dataset to new file for storytelling and visualization

# diabetes.to_csv('diabetes_cleaned.csv')